In [18]:
#ONLY RUN THIS FILE ONCE! get_all_objects() is slow... luckily I've saved the resulting dataframe as a .pkl

import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
stemmer = ps()
from nltk.stem.cistem import Cistem
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import plotly.express as px
import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [7]:
#this function makes the api call to the server
def api_call(x):
    session = requests.Session()
    data_list = []
    parameters = {"items_per_page": 25,
                      "page": x}
    response = session.get("https://risdmuseum.org/api/v1/collection", params=parameters).json()
    for item in response:
        data_list.append(item)
    return data_list 
    
            

In [3]:
#Multithreading to speed up API calls
def runner(function, var):
    giant_list = []
    threads= []
    with ThreadPoolExecutor(max_workers=20) as executor:
        for x in range(0, var):
            threads.append(executor.submit(function, x))
            
        for task in as_completed(threads):
            giant_list.append(task.result())
    return giant_list

In [4]:
data_list = runner(api_call, 3289)

In [8]:
#Normalizes JSON objects and filters for UNKNOWN MAKER 
def dict_to_df(a_dict):
    new_df = pd.json_normalize(a_dict)
    new_df['makers'] = new_df['makers'].astype(str)
    filtered_df = new_df[new_df['makers'].str.contains("'name': 'Unknown'")]
    return filtered_df

In [14]:
def dict_to_df_known(a_dict):
    new_df = pd.json_normalize(a_dict)
    new_df['makers'] = new_df['makers'].astype(str)
    filtered_df = new_df[~new_df['makers'].str.contains("'name': 'Unknown'")]
    return filtered_df

In [9]:
#Gets full data
def get_dict(a_dict):
    return pd.json_normalize(a_dict)

In [10]:
#Multithreading for the dict operation
def dict_runner(function, var):
    df = pd.DataFrame()
    threads= []
    with ThreadPoolExecutor(max_workers=20) as executor:
        for a_dict in var:
            threads.append(executor.submit(function, a_dict))
            
        for task in as_completed(threads):
            df = df.append(task.result())
    return df

In [25]:
object_df = dict_runner(dict_to_df, data_list)

In [27]:
object_df.to_pickle("./unknown_makers_FINAL.pkl")

In [11]:
full_df = dict_runner(get_dict, data_list)

In [12]:
full_df.to_pickle("./full_dataset_FINAL.pkl")

In [15]:
full_known_df = dict_runner(dict_to_df_known, data_list)

In [19]:
print(full_known_df['makers'])

0     [{'name': 'Duane Slick', 'risdAffiliation': '(...
1     [{'name': 'Abraham Walkowitz', 'risdAffiliatio...
2     [{'name': 'Josiah Wedgwood & Sons', 'risdAffil...
3     [{'name': 'Marcella Kroll', 'risdAffiliation':...
4     [{'name': 'Ryan Lesser', 'risdAffiliation': '(...
5     [{'name': 'Nagrom Monceaux', 'risdAffiliation'...
6     [{'name': 'Robin Nanney', 'risdAffiliation': '...
7     [{'name': 'Jenny Nichols', 'risdAffiliation': ...
8     [{'name': 'Mark Pedini', 'risdAffiliation': No...
9     [{'name': 'Jonathan Wisehart', 'risdAffiliatio...
10    [{'name': 'Pippi Zornoza', 'risdAffiliation': ...
11    [{'name': 'Harry Callahan', 'risdAffiliation':...
12    [{'name': 'Luis F. Romero', 'risdAffiliation':...
13    [{'name': 'Sigmar Polke', 'risdAffiliation': N...
14    [{'name': 'Sigmar Polke', 'risdAffiliation': N...
15    [{'name': 'Sigmar Polke', 'risdAffiliation': N...
16    [{'name': 'Max Klinger', 'risdAffiliation': No...
17    [{'name': 'Max Klinger', 'risdAffiliation'

In [20]:
full_known_df.to_pickle("./full_known_dataset_FINAL.pkl")